In [8]:
# Enables figures loading outside of browser.
# If not run, figures will load inline.
%matplotlib

import os
import math
import pandas as pd
import numpy as np
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import matplotlib.ticker
import datetime
import collections
import itertools

from scipy import stats

# Depends on: pip install sklearn
from sklearn.model_selection import train_test_split

# Some matplotlib features are version dependent.
assert(matplotlib.__version__ >= '2.1.2')

# Depends on: pip install --upgrade google-cloud-bigquery
from google.cloud import bigquery

def run_query(query, project='mlab-sandbox'):
    client = bigquery.Client(project=project)
    job = client.query(query)

    results = collections.defaultdict(list)
    for row in job.result(timeout=3000):
        for key in row.keys():
            results[key].append(row.get(key))

    return pd.DataFrame(results)

def unlog(x, pos):
    """Formats the x axis for histograms taken on the log of values."""
    v = math.pow(10, x)
    frac, whole = math.modf(v)
    if frac > 0:
        return '%.1f' % v
    else:
        return '%d' % whole
    
    
def hist(vals, bin_count, log=True, cdf=False):
    """Produces hist or cdf values for smooth plots."""
    if log:
        r = [math.log10(x) for x in vals]
    else:
        r = vals
        
    m, bins = np.histogram(r, bin_count, normed=True)
    m = m.astype(float)

    tops = m
    if cdf:
        tops = np.cumsum(m)
        total = sum(m)
        tops = [float(t) / total for t in tops ]
    
    return tops, bins


logFormatter = matplotlib.ticker.FuncFormatter(unlog)

Using matplotlib backend: MacOSX


In [9]:
def plot_df(
    df, xname='', yname='',
    cname='', bins=None, cdf=False,
    fig_by='', axes_by='', group_by='',
    figsize=(6,8), axes=(1,1),
    label='{group}',
    xlabel='', ylabel='',
    xlim=(), ylim=(),
    fx=list, fy=list,
    xlog=False, ylog=False,
    suptitle='', title='', legend={}, figmap=None, log=None, fxn=None):
    """
    Args:
        df: pandas.DataFrame,
        xname: str, name of column to use as x-axis.
        yname: str, name of column to use as y-axis.
        cname: str, name of column to use as data source.
        cdf: bool,
        bins: int or callable,
        fig_by: str, name of column to split data into multiple figures.
        axes_by: str, name of column to arrange into a single panel.
        group_by: str, name of column to plot common split_by and group_by columns.
        figsize: (int, int), dimensions of figure.
        axes: (int, int), arrangement of axes within figure.
        label: str,
        xlabel: str,
        ylabel: str,
        xlim: (xmin, xmax),
        ylim: (ymin, ymax),
        fx: func,
        fy: func,
        xlog: bool,
        ylog: bool,
        suptitle: str,
        title: str,
        legend: **legend_args,
        figmap: returned from a previous run of plot_df, used to overlay values
            from multiple data frames. Must use the same fig_by, axes_by, and
            group_by values.
        log: bool,
        f: callable,
    Returns:
      dict of str to (figures, axes) tuples
    """
    def info(f):
        if log:
            print f

    if figmap is None:
        info('new figmap')
        figmap = {}
    scatter = None
    if (xname and yname):
        scatter = True
    if cname:
        scatter = False
    if scatter is None:
        raise Exception('Provide xname and yname or cname')
    
    for f in sorted(set(['default'] if not fig_by else df[fig_by])):
        if f in figmap:
            info('reusing figmap for %s' % f)
            fig, ax = figmap[f]
        else:
            fig = plt.figure(figsize=figsize)
            ax = fig.subplots(axes[0], axes[1], squeeze=False)
            info('saving figmap for %s' % f)
            figmap[f] = (fig, ax)
        ax_index = list(itertools.product(range(axes[0]), range(axes[1])))
        
        df_fig = df if f == 'default' else df[df[fig_by] == f]
        for p, a in enumerate(sorted(set(['default'] if not axes_by else df_fig[axes_by]))):

            df_axes = df_fig if a == 'default' else df_fig[df_fig[axes_by] == a]
            if p >= len(ax_index):
                print 'SKIPPING', p, f, a, 'too few axes positions'
                continue
                
            i, j = ax_index[p]
            for g in sorted(set(['default'] if not group_by else df_axes[group_by])):
                df_g = df_axes if g == 'default' else df_axes[df_axes[group_by] == g]

                if scatter:
                    x = fx(df_g[xname])
                    y = fy(df_g[yname])
                    l = label.format(figure=f, axis=a, group=g)
                    ax[i][j].scatter(x, y, s=1, label=l)
                else:
                    r = df_g[cname]
                    if bins is None:
                        size = int(math.sqrt(len(r)))
                    else:
                        size = bins(r)
                    if fxn:
                        fxn(r, figure=f, axis=a, group=g, size=size)
                    info("%s %s %s %s %s" % (f, a, g, size, len(r)))
                    h_tops, h_bins = hist(r, size, log=xlog , cdf=cdf)
                    l = label.format(figure=f, axis=a, group=g, size=size)
                    ax[i][j].plot(h_bins[:-1], h_tops, label=l)

            if i != len(ax)-1:
                ax[i][j].set_xticklabels([])

            if title:
                ax[i][j].set_title(title.format(figure=f, axis=a, group=g))
            if ylabel:
                ax[i][j].set_ylabel(ylabel.format(figure=f, axis=a, group=g))
            if xlabel:
                ax[i][j].set_xlabel(xlabel.format(figure=f, axis=a, group=g))

            if xlim:
                ax[i][j].set_xlim(xlim)
            if ylim:
                ax[i][j].set_ylim(ylim)

            ax[i][j].grid(color='#dddddd')
            ax[i][j].legend(fontsize='x-small', **legend)
            if scatter:
                ax[i][j].tick_params(axis='x', labelrotation=-90)
            if xlog:
                ax[i][j].xaxis.set_major_formatter(logFormatter)
            if ylog:
                ax[i][j].semilogy()

        if suptitle:
            fig.suptitle(suptitle.format(figure=f))
        fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    return figmap

        
def plot_scatter(df, xname, yname, **kwargs):
    return plot_df(df, xname=xname, yname=yname, **kwargs)

    
def plot_hist(df, cname, bins=None, **kwargs):
    return plot_df(df, cname=cname, bins=bins, **kwargs)

# UPLINK UTILIZATION OVER TIME

In [10]:
df_disco_pct = run_query("""
#standardSQL
SELECT

  UPPER(REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*')) AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,  
  0.8 * APPROX_QUANTILES(value, 101)[ORDINAL(50)] as bytes_50th,
  APPROX_QUANTILES(value, 101)[ORDINAL(90)] as bytes_90th

FROM (
  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab1.[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
    metric LIKE 'switch.octets.uplink.tx'
    AND REGEXP_CONTAINS(hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  GROUP BY
    hostname, metric, sts, value
)
WHERE
  hostname IS NOT NULL
GROUP BY
  hostname, day, ts
ORDER BY
  hostname, day, ts
""")

In [11]:
plot_scatter(
    df_disco_pct, 'ts', 'bytes_50th',
    axes_by='metro', group_by='site', axes=(3, 1),
    suptitle='Daily Median Uplink Utilization',
    ylabel="Mbps",
    title='{axis}',
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e4, 1e9),
    fx=lambda l: [pd.to_datetime(t, unit='s') for t in l],
    legend={'loc':3, 'ncol':7, 'columnspacing':1},
    ylog=True)

{'default': (<matplotlib.figure.Figure at 0x1a191710d0>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a190f0b50>],
        dtype=object))}

# Daily DISCO discard ratios

In [12]:
df_disco_ratio = run_query("""
WITH measurementlab_switch_dedup AS (
  SELECT
    metric,
    REGEXP_EXTRACT(hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
    sample.timestamp AS sts,
    sample.value AS value
  FROM
    `measurement-lab.base_tables.switch*`,
    UNNEST(sample) AS sample
  WHERE
    (metric LIKE 'switch.discards.uplink.tx' OR metric LIKE 'switch.unicast.uplink.tx')
    AND REGEXP_CONTAINS(hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  GROUP BY
    hostname, metric, sts, value
)

SELECT
  UPPER(REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3})[0-9]{2}.*')) AS metro,
  REGEXP_EXTRACT(hostname, r'mlab1.([a-z]{3}[0-9]{2}).*') AS site,
  hostname,
  day,
  ts,
  IF(total > 0, discards / total, 0) as ratio
FROM (
SELECT
  hostname,
  FORMAT_TIMESTAMP("%Y-%m-%d", TIMESTAMP_TRUNC(sts, DAY)) AS day,
  UNIX_SECONDS(TIMESTAMP_TRUNC(sts, DAY)) AS ts,
  SUM(IF(metric = "switch.discards.uplink.tx", value, 0)) AS discards,
  SUM(IF(metric = "switch.unicast.uplink.tx", value, 0)) AS total
FROM
  measurementlab_switch_dedup
WHERE
  hostname IS NOT NULL
GROUP BY
  hostname, day, ts
HAVING
  discards < total
ORDER BY
  hostname, day, ts
)
GROUP BY
  hostname, day, ts, ratio
HAVING
  ratio < 0.01
ORDER BY
  hostname, day, ts
""")

In [13]:
plot_scatter(
    df_disco_ratio, 'ts', 'ratio', axes_by='metro', group_by='site', axes=(3, 1),
    suptitle='Daily Packet Loss Ratio (discards / unicast)',
    ylabel="Ratio",
    title='{axis}',
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e-6, 1e-1),
    fx=lambda l: [pd.to_datetime(t, unit='s') for t in l],
    legend={'loc':2},
    ylog=True)

{'default': (<matplotlib.figure.Figure at 0x1a1966d4d0>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a19641b90>],
        dtype=object))}

# NDT Median Download Rates

In [14]:
df_ndt_all = run_query("""
WITH mlab_ndt AS (
  SELECT
    UPPER(REGEXP_EXTRACT(connection_spec.server_hostname, r"([a-z]{3})[0-9]{2}")) as metro,
    REGEXP_EXTRACT(connection_spec.server_hostname, r"([a-z]{3}[0-9]{2})") as site,
    web100_log_entry.connection_spec.remote_ip as remote_ip,
    log_time,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps

  FROM
    `measurement-lab.release.ndt_all`

  WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"(lga|dfw|nuq)\d\d")
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND connection_spec.data_direction = 1
    AND web100_log_entry.connection_spec.remote_ip != "45.56.98.222"
    AND web100_log_entry.connection_spec.remote_ip != "2600:3c03::f03c:91ff:fe33:819"
    AND web100_log_entry.connection_spec.remote_ip != "35.225.75.192"
    AND web100_log_entry.connection_spec.remote_ip != "35.192.37.249"
    AND web100_log_entry.connection_spec.remote_ip != "35.193.254.117"
    AND anomalies.no_meta is not true
  
  GROUP BY
    connection_spec.server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps
)
    
SELECT
  metro,
  site,
  day,
 --  AVG(download_mbps) as download_mbps,
  APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps,
  count(*) as count
FROM
(
  SELECT
    metro,
    site,
    TIMESTAMP_TRUNC(log_time, DAY) as day,
    -- APPROX_QUANTILES(download_mbps, 101)[ORDINAL(50)] as download_mbps
    MAX(download_mbps) as download_mbps
  FROM
    mlab_ndt

  GROUP BY
    metro, site, day, remote_ip
)

GROUP BY
  metro, site, day

ORDER BY
  day
""")


In [15]:
plot_scatter(
    df_ndt_all, 'day', 'download_mbps', axes_by='metro', group_by='site', axes=(3, 1),
    suptitle='Median NDT Download Rates',
    ylabel="Mbps",
    title='{axis}',
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(0, 50),
    fx=lambda l: [pd.to_datetime(t) for t in l],
    legend={'loc':3, 'ncol':7, 'columnspacing':1})

{'default': (<matplotlib.figure.Figure at 0x1a197469d0>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a1a3ff050>],
        dtype=object))}

# NDT Segs Retrans

In [16]:
# NOT ENOUGH HISTORICAL NDT DATA TO GET FULL TIMELINE.

In [17]:
df_ndt_retrans = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as hostname,
    web100_log_entry.connection_spec.remote_ip as remote_ip,
    log_time,
    web100_log_entry.snap.SegsRetrans as SegsRetrans,
    web100_log_entry.snap.SegsOut as SegsOut

  FROM
    `measurement-lab.release.ndt_all`

  WHERE
        REGEXP_CONTAINS(connection_spec.server_hostname, r"(lga|dfw|nuq)\d\d")
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND connection_spec.data_direction = 1
    AND web100_log_entry.connection_spec.remote_ip != "45.56.98.222"
    AND web100_log_entry.connection_spec.remote_ip != "2600:3c03::f03c:91ff:fe33:819"
    AND web100_log_entry.connection_spec.remote_ip != "35.225.75.192"
    AND web100_log_entry.connection_spec.remote_ip != "35.192.37.249"
    AND web100_log_entry.connection_spec.remote_ip != "35.193.254.117"
    AND log_time >= TIMESTAMP("2016-06-01")
  
  GROUP BY
    connection_spec.server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    SegsRetrans,
    SegsOut
)
    
SELECT
  UPPER(REGEXP_EXTRACT(hostname, r"([a-z]{3})[0-9]{2}")) as metro,
  REGEXP_EXTRACT(hostname, r"([a-z]{3}[0-9]{2})") as site,
  day,
  APPROX_QUANTILES(ratio, 101)[ORDINAL(50)] AS median_ratio,
  count(*) as count
FROM
(
  SELECT
    hostname,
    TIMESTAMP_TRUNC(log_time, DAY) as day,
    MAX(SAFE_DIVIDE(SegsRetrans, SegsOut)) as ratio

  FROM
    mlab_ndt

  GROUP BY
    hostname,
    day,
    remote_ip
)

GROUP BY
  hostname, day

ORDER BY
  day
""")

In [18]:
plot_scatter(
    df_ndt_retrans, 'day', 'median_ratio', axes_by='metro', group_by='site', axes=(3, 1),
    suptitle='Median NDT Retransmission Ratio - (SegsRetran / SegsOut)',
    ylabel="Ratio",
    title='{axis}',
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    ylim=(1e-6, 1e-1),
    fx=lambda l: [pd.to_datetime(t) for t in l],
    legend={'loc':2},
    ylog=True)

{'default': (<matplotlib.figure.Figure at 0x1a193b3290>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a1966d850>],
        dtype=object))}

# NDT test distributions - Before & After

In [19]:
df_ndt_variance = run_query("""
WITH mlab_ndt AS (
  SELECT
    connection_spec.server_hostname as server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip AS remote_ip,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps
  FROM
    `measurement-lab.base_tables.ndt*`
  WHERE

  (    TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
    OR TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25"))
  AND REGEXP_CONTAINS(connection_spec.server_hostname, r"mlab1.(dfw|lga|nuq)\d\d")
  AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
  AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
  AND connection_spec.data_direction = 1
  
  GROUP BY
    server_hostname,
    log_time,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps)


SELECT
  UPPER(REGEXP_EXTRACT(server_hostname, r'mlab[1-4].([a-z]{3})[0-9]{2}.*')) AS metro,
  REGEXP_EXTRACT(server_hostname, r'mlab[1-4].([a-z]{3}[0-9]{2}).*') AS site,
  REGEXP_EXTRACT(server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
  CASE
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25") THEN 'before-2w'
    WHEN TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25") THEN 'after-2w'
    ELSE 'what'
  END AS period,
  remote_ip,
  STDDEV(download_mbps) AS download_stddev,
  (STDDEV(download_mbps) / AVG(download_mbps)) AS download_cv,
  MAX(download_mbps) AS download_max,
  MIN(download_mbps) AS download_min,
  AVG(download_mbps) AS download_avg

FROM
  mlab_ndt
WHERE
  remote_ip IN(
    SELECT
      remote_ip
    FROM (
      SELECT
        remote_ip, count(*) as c1
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-02-11") AND TIMESTAMP("2018-02-25")
      GROUP BY
        remote_ip
      HAVING c1 > 5
    ) INNER JOIN (
      SELECT
        remote_ip AS remote_ip, count(*) as c2
      FROM
        mlab_ndt
      WHERE
        TIMESTAMP_TRUNC(log_time, DAY) BETWEEN TIMESTAMP("2018-03-11") AND TIMESTAMP("2018-03-25")
      GROUP BY
        remote_ip
      HAVING c2 > 5
    ) USING (remote_ip)) 
GROUP BY
  server_hostname,
  period,
  remote_ip
  --download_mbps

HAVING download_stddev is not NULL
""")

In [31]:
values = {}
def ks_compare(r, figure='', axis='', group='', size=''):
    values["%s-%s-%s" % (figure, axis, group)] = r
    if group == 'before-2w':
        after = values["%s-%s-%s" % (figure, axis, 'after-2w')]
        result = stats.ks_2samp(r, after)
        if result.pvalue < 0.01:
            print 'diff', figure, axis, result

            
f = plot_hist(
    df_ndt_variance, 'download_max', lambda r: int(math.sqrt(len(r))),
    fig_by='metro', axes_by='site', group_by='period',
    suptitle='Distribution of NDT Downloads - MAX(per remote_ip)',
    label='{group} ({size})',
    title='{axis}', axes=(3, 2),
    xlim=(math.log10(.01), math.log10(1000)),
    cdf=False, xlog=True, figsize=(9, 7),
    fxn=ks_compare)

diff LGA lga02 Ks_2sampResult(statistic=0.13487760908755242, pvalue=6.455805572283752e-05)
diff LGA lga03 Ks_2sampResult(statistic=0.05627438029592241, pvalue=0.0006379908963118555)
diff LGA lga07 Ks_2sampResult(statistic=0.049918312849347335, pvalue=0.0061856763535876064)


In [32]:
values = {}
def ks_compare(r, figure='', axis='', group='', size=''):
    values["%s-%s-%s" % (figure, axis, group)] = r
    if group == 'before-2w':
        after = values["%s-%s-%s" % (figure, axis, 'after-2w')]
        result = stats.ks_2samp(r, after)
        if result.pvalue < 0.01:
            print 'diff', figure, axis, result

f = plot_hist(
    df_ndt_variance, 'download_avg', lambda r: int(math.sqrt(len(r))),
    fig_by='metro', axes_by='site', group_by='period',
    suptitle='Distribution of NDT Downloads - AVERAGE(per remote_ip)',
    label='{group} ({size})',
    title='{axis}', axes=(3, 2),
    xlim=(math.log10(.01), math.log10(1000)),
    cdf=True, xlog=True, figsize=(9, 7),
    fxn=ks_compare)

diff LGA lga03 Ks_2sampResult(statistic=0.06594182035779406, pvalue=3.1651607393831e-05)
diff LGA lga07 Ks_2sampResult(statistic=0.05214086679603924, pvalue=0.0036553957015487497)


In [22]:
import time
def query(site):
    print 'running query', site, time.ctime()
    return """
CREATE TEMPORARY FUNCTION
  timeBin(ts_usec INT64,
    size INT64) AS ( CAST(TRUNC(ts_usec / 1e6 / 10) * 10 AS INT64) );

WITH
  mlab_ndt_dedup AS (
  SELECT
    test_id,
    log_time,
    connection_spec.server_hostname AS hostname,
    web100_log_entry.snap.StartTimeStamp as StartTimeStamp,
    (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps

  FROM
    `measurement-lab.release.ndt_all`
  WHERE
          log_time BETWEEN TIMESTAMP("2016-06-01") AND TIMESTAMP("2018-08-01")
      AND (connection_spec.server_hostname = "mlab1."""+site+""".measurement-lab.org" OR connection_spec.server_hostname = "ndt.iupui.mlab1."""+site+""".measurement-lab.org")
      AND connection_spec.data_direction = 1
      AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
      AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
      AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000


  GROUP BY
    test_id,
    log_time,
    hostname,
    StartTimeStamp,
    web100_log_entry.connection_spec.remote_ip,
    web100_log_entry.connection_spec.local_ip,
    web100_log_entry.connection_spec.remote_port,
    web100_log_entry.connection_spec.local_port,
    download_mbps)
    
  ,ndt_test_ids_with_discards AS (
  SELECT
    ndt.test_id as test_id,
    ndt.hostname as hostname,
    ndt.day as day,
    SUM(disco.discards) AS discards,
    ndt.download_mbps as download_mbps
  FROM (
    SELECT
      hostname,
      UNIX_SECONDS(sample.timestamp) - 10 AS tstart,
      UNIX_SECONDS(sample.timestamp) AS tend,
      sample.value AS discards
    FROM
      `measurement-lab.base_tables.switch*`,
      UNNEST(sample) AS sample
    WHERE
      metric = 'switch.discards.uplink.tx'
      AND sample.timestamp BETWEEN TIMESTAMP("2016-06-01") AND TIMESTAMP("2018-08-01")
      AND hostname = "mlab1."""+site+""".measurement-lab.org"
    GROUP BY
      hostname,
      tstart,
      tend,
      discards
    HAVING
      discards > 0
  ) AS disco
  JOIN (
    SELECT
      test_id,
      REGEXP_EXTRACT(connection_spec.server_hostname, r"(mlab1."""+site+""".measurement-lab.org)") as hostname,
      TIMESTAMP_TRUNC(log_time, DAY) as day,
      timeBin(web100_log_entry.snap.StartTimeStamp, 10) AS tstart,
      timeBin(web100_log_entry.snap.StartTimeStamp, 10) + 20 AS tend,
      (8 * (web100_log_entry.snap.HCThruOctetsAcked / (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd))) AS download_mbps

  FROM
    `measurement-lab.release.ndt_all`
    --`measurement-lab.base_tables.ndt*`
  WHERE
          log_time BETWEEN TIMESTAMP("2016-06-01") AND TIMESTAMP("2018-08-01")
      AND connection_spec.data_direction = 1
      AND (connection_spec.server_hostname = "mlab1."""+site+""".measurement-lab.org" OR connection_spec.server_hostname = "ndt.iupui.mlab1."""+site+""".measurement-lab.org")
      AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
      AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
      AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000

    GROUP BY
      test_id,
      hostname,
      day,
      tstart,
      tend,
      download_mbps ) AS ndt
  ON (disco.hostname = ndt.hostname
      AND (disco.tstart = ndt.tstart OR disco.tend = ndt.tend))
  GROUP BY
    day, hostname, test_id, download_mbps
  )


SELECT
  day, metro, site, hostname, discards, COUNT(*) as count
FROM
(
  SELECT
    TIMESTAMP_TRUNC(log_time, DAY) as day,
    UPPER(REGEXP_EXTRACT(connection_spec.server_hostname, r'mlab[1-4].([a-z]{3})[0-9]{2}.*')) AS metro,
    REGEXP_EXTRACT(connection_spec.server_hostname, r'mlab[1-4].([a-z]{3}[0-9]{2}).*') AS site,
    REGEXP_EXTRACT(connection_spec.server_hostname, r'(mlab[1-4].[a-z]{3}[0-9]{2}).*') AS hostname,
    CASE
      WHEN test_id IN(SELECT test_id from ndt_test_ids_with_discards) THEN 'non-zero'
      ELSE 'zero'
--      WHEN test_id NOT IN(SELECT test_id from ndt_test_ids_with_discards) THEN 'without'
--      ELSE 'what'
    END as discards

  FROM
    --`measurement-lab.base_tables.ndt*`
    `measurement-lab.release.ndt_all`
  WHERE
        log_time BETWEEN TIMESTAMP("2016-06-01") AND TIMESTAMP("2018-08-01")
    AND connection_spec.data_direction = 1
    AND (connection_spec.server_hostname = "mlab1."""+site+""".measurement-lab.org" OR connection_spec.server_hostname = "ndt.iupui.mlab1."""+site+""".measurement-lab.org")
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) >= 9000000
    AND (web100_log_entry.snap.SndLimTimeRwin + web100_log_entry.snap.SndLimTimeCwnd + web100_log_entry.snap.SndLimTimeSnd) < 600000000
    AND web100_log_entry.snap.HCThruOctetsAcked >= 1000000

  )
GROUP BY
  day, metro, site, hostname, discards
"""

if False:
    df_test_counts = pd.concat([
    run_query(query("dfw01")),
    run_query(query("dfw02")),
    run_query(query("dfw03")),
    run_query(query("dfw04")),
    run_query(query("dfw05")),
    run_query(query("dfw06")),
    run_query(query("lga02")),
    run_query(query("lga03")),
    run_query(query("lga04")),
    run_query(query("lga05")),
    run_query(query("lga06")),
    run_query(query("lga07")),
])

running query dfw01 Thu Aug 30 21:15:41 2018
running query dfw02 Thu Aug 30 21:16:08 2018
running query dfw03 Thu Aug 30 21:20:34 2018
running query dfw04 Thu Aug 30 21:21:45 2018
running query dfw05 Thu Aug 30 21:23:38 2018
running query dfw06 Thu Aug 30 21:26:26 2018
running query lga02 Thu Aug 30 21:27:09 2018
running query lga03 Thu Aug 30 21:27:52 2018
running query lga04 Thu Aug 30 21:29:04 2018
running query lga05 Thu Aug 30 21:29:32 2018
running query lga06 Thu Aug 30 21:30:01 2018
running query lga07 Thu Aug 30 21:32:12 2018


In [30]:
# NOTE: does not preserve binsize across group_by. Each line re-calculates the bin size.
print df_test_counts.keys()
plot_scatter(
    df_test_counts, 'day', 'count',
    fig_by='metro', axes_by='site', group_by='discards',
    suptitle='NDT Test Counts (with or without discards)',
    label='{group}',
    title='{axis}',
    axes=(3, 2), figsize=(12, 10),
    ylim=(-200, 30000),
    xlim=(pd.to_datetime("2016-05-31"), pd.to_datetime("2018-08-01")),
    fx=lambda l: [pd.to_datetime(t) for t in l])

Index([u'count', u'day', u'discards', u'hostname', u'metro', u'site'], dtype='object')


{u'DFW': (<matplotlib.figure.Figure at 0x1a345d2890>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a342ec290>,
        dtype=object)),
 u'LGA': (<matplotlib.figure.Figure at 0x1a33082a90>,
  array([[<matplotlib.axes._subplots.AxesSubplot object at 0x1a2a7ea910>,
        dtype=object))}